**handson用資料としての注意点**

普通、同じセル上で何度も試行錯誤するので、最終的に上手くいったセルしか残らず、失敗したセルは残りませんし、わざわざ残しません。

今回はhandson用に 試行・思考過程を残したいと思い、エラーやミスが出ても下のセルに進んで処理を実行するようにしています。

notebookのセル単位の実行ができるからこそのやり方かもしれません。良い。

（下のセルから文は常体で書きます。）

kunai (@jdgthjdg)

---


# ここまでの処理を整理して、2008〜2019のデータを繋いでみる

## xls,xlsxファイルを漁る

In [ ]:
from pathlib import Path

In [ ]:
base_dir = Path("../../../data")
base_dir.exists()

In [ ]:
list(base_dir.glob("*_kansai/*"))

In [ ]:
p = list(base_dir.glob("*_kansai/*"))[0]
p.name

In [ ]:
kansai_kafun_files = []
for p in base_dir.glob("*_kansai/*"):
    # AMeDASだけ弾く
    if not p.name.startswith("AMeDAS"):
        kansai_kafun_files.append(p)
kansai_kafun_files

lock ファイルが混じってしまった。<BR>
AMeDASだけ弾くと .lockファイルも入ってしまうので、<br>
試しに **読めない文字 ( ë╘ò▓âfü[â )** で引っ掛けてみる

In [ ]:
kansai_kafun_files = []
for p in base_dir.glob("*_kansai/*"):
    # AMeDASだけ弾くと .lockファイルも入ってしまうので、読めない謎の文字で引っ掛けてみる
    if p.name.startswith("ë╘ò▓âfü[â"):
        kansai_kafun_files.append(p)
kansai_kafun_files

いけた

ソートしてもいいけど、どのみち日付データはとるのでこのまま

---

# 今までの処理を適用していく

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#設定でDataFrameなどが長く表示されないようにします（画面領域の消費を抑えてhandsonをしやすくするため）
# 長い場合の途中の省略表示(...)を出す閾値の設定（折り返しとは無関係）
pd.set_option('max_rows',10)
pd.set_option('max_columns',20) # これを超えたら全部は表示しない。 A B C ... X Y Z のように途中を省く。

In [ ]:
p = kansai_kafun_files[-1]
print(p) # for の残りかす
df = pd.read_excel(p, skiprows=1).iloc[:,:-2]
df

In [ ]:
str_concat_h0_23 = df["年"].astype(str)+"/"+df["月"].astype(str)+"/"+df["日"].astype(str)+"/"+(df["時"]-1).astype(str) # 時から1引いてる
df["date_hour"] = pd.to_datetime(str_concat_h0_23, format="%Y/%m/%d/%H")
df.set_index("date_hour", inplace=True)
df = df.drop(columns=["年","月","日","時",]) # こっちでも全然良い
df

# ここまでを関数にする

多くの試行錯誤があったがこれだけのコードに圧縮された・・・

In [ ]:
def load_kafun_excel(path):
    df = pd.read_excel(path, skiprows=1).iloc[:,:-2]
    str_concat_h0_23 = df["年"].astype(str)+"/"+df["月"].astype(str)+"/"+df["日"].astype(str)+"/"+(df["時"]-1).astype(str) # 時から1引いてる
    df["date_hour"] = pd.to_datetime(str_concat_h0_23, format="%Y/%m/%d/%H")
    df.set_index("date_hour", inplace=True)
    df = df.drop(columns=["年","月","日","時",]) # こっちでも全然良い
    return df

In [ ]:
load_kafun_excel(p)

## for文で回す 安心の有限

In [ ]:
kafun_df_list = []
for p in kansai_kafun_files:
    df = load_kafun_excel(p)
    kafun_df_list.append(df)

In [ ]:
kafun_df_list[0]

# リスト内のdfを行方向(縦方向, y方向)に連結する

**[pd.concat](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html)**

df の連結/結合/マージ  
http://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html


In [ ]:
kafun = pd.concat(kafun_df_list, axis=1)
kafun.shape

In [ ]:
kafun.columns

<br>
ミスってcolumnsの数が成長した。横方向につながっている  
<br>
<br>

In [ ]:
kafun = pd.concat(kafun_df_list, axis=0)  # Warning

warning された。 今後ソートしないとのこと

In [ ]:
kafun = pd.concat(kafun_df_list, axis=0, sort=False)
kafun.shape

このaxis方向でも columns が倍くらいに増えている・・・

In [ ]:
kafun.columns

恐らく年が変わった時に列の名前が変わったようだ（多分担当者も）   
データのフォーマットが変わると、恐ろしく面倒なことが起きる・・・

In [ ]:
kafun_df_list[0].columns

In [ ]:
kafun_df_list[1].columns

# 想像以上に全然違う・・・

ファイル名をみる

In [ ]:
kansai_kafun_files[0].name

In [ ]:
kansai_kafun_files[1].name

xlsとxlsxの頃から変わったのかも？　とりあえず ロードした時にcolumnsを表示する雑な関数を作って試す

In [ ]:
def show_columns(path):
    df = pd.read_excel(path, skiprows=1).iloc[:,:-2]
    return df.columns

In [ ]:
for p in kansai_kafun_files:
    print(p, show_columns(p))

<br>
年が順不同で見にくいので結局ソートしてみる

In [ ]:
sorted(kansai_kafun_files) 

---
**もしソートがうまくいかないパスだったら sorted の key=を設定する**

In [ ]:
p.name[10:14]  # ファイル名から年を抜き出すスライスがこれだった

In [ ]:
# フォルダ名にかかわらず、ファイル名の 20xx で数値のソートされる
sorted(kansai_kafun_files, key=lambda x:int(x.name[10:14]))

In [ ]:
sorted(kansai_kafun_files, key=lambda x: (-1)*int(x.name[10:14]))  # マイナスにすれば逆になるのが分かる

<br>
ソート後にまた columns を見る

In [ ]:
for p in sorted(kansai_kafun_files):
    print(p, show_columns(p))

---
2015年にxlsxに変わった途端に・・・<br>

これでは元データのエクセルか、ウェブページの注釈的なものを探さないと追跡ができない  

もういちどエクセルを見てみると、  
古いデータには、 "地点" という別のシートがあった！

sheet_nameには文字列以外にもindexが使えるとdocumentに書いてあった。  
誰かのブログをコピってるだけだったら気づけなかったかもしれない。（自戒）

In [ ]:
names = pd.read_excel("../../../data/2008_kansai/ë╘ò▓âfü[â^2008(è╓É╝).xls", sheet_name=1)
names

## 列名をrenameするmapperを作る

ここからは適当にメソッドを探して対処していくしかない。。  
pandas力が試される・・・

mappingするなら辞書が良いから辞書っぽいのを探す

In [ ]:
names["地点名"].to_dict()

<br>
key:value が index:列の値 となるdictができたので、index を "地点名" 列にして、"施設名" との .to_dictすれば良さそう

In [ ]:
rename_mapper = names.set_index("地点名")["施設名"].to_dict()
rename_mapper

これ。きた。

In [ ]:
df.rename(columns=rename_mapper).head(1)
# OK

関数に埋め込む

In [ ]:
def load_kafun_excel_renamed_columns(path):
    df = pd.read_excel(path, skiprows=1).iloc[:,:-2]
    try:
        name = pd.read_excel(path, sheet_name=1)
        rename_mapper = names.set_index("地点名")["施設名"].to_dict()
        df = df.rename(columns=rename_mapper)
        
    except Exception as e:
        print(path, e)
    
    str_concat_h0_23 = df["年"].astype(str)+"/"+df["月"].astype(str)+"/"+df["日"].astype(str)+"/"+(df["時"]-1).astype(str) # 時から1引いてる
    df["date_hour"] = pd.to_datetime(str_concat_h0_23, format="%Y/%m/%d/%H")
    df.set_index("date_hour", inplace=True)
    df = df.drop(columns=["年","月","日","時",]) # こっちでも全然良い
    return df

In [ ]:
kafun_df_list = []
for p in sorted(kansai_kafun_files):
    df = load_kafun_excel_renamed_columns(p)
    kafun_df_list.append(df)
kafun_renamed = pd.concat(kafun_df_list, axis=0, sort=False)
kafun_renamed.shape

xlsxだけエラーになってくれてるのでxlsでは読み込めているようだ

果たして結果は？

In [ ]:
kafun_renamed.columns

---
似た名前を探すためにソートしてみる

In [ ]:
kafun_renamed.columns.sort_values()

---

**'北山緑化植物園','北山緑化植物園（西宮市都市整備公社）'**

**'西播磨', '西播磨県民局西播磨総合庁舎'** とか同一では？  
列名のゆらぎが・・・（予想はしていたがこれを全部追うのは大変なので今回はパス！）

ここでHPをもう一度見てみると。  
http://kafun.taiki.go.jp/library.html#4

>彦根地方気象台	彦根市城町２丁目５－２５	彦根	平成29年度に彦根市役所から移設  
>舞鶴市西コミュニティセンター	舞鶴市字南田辺１番地	舞鶴	平成29年度に京都府中丹東保健所より移設


## 追っかけるのも大変、かつ、そこまでを求めていないため、今回は少ないデータも全部残して次へ進む

---

## （今回はしないが）もし少ないものを弾きたいなら

全部の対応を探すのは流石に厳しそうなので、各列での NaNの値を数えてみて、NaNの値が少ないものは2008〜2018まで列名がつながっていると判断する

**count()** がそれに当たる  
**sort_values** でソートしている

In [ ]:
kafun_renamed.count().sort_values(ascending=True).head(10)

In [ ]:
kafun_renamed.count().sort_values(ascending=False).head(10) # Falseにしなくても、上のコードをtailにするだけでも良い

---

# 一旦ここまでのデータをpickleに保存

ここまでの処理で生成したDataFrameをpickleで保存しておく。  
pickle にしておくと読み込みも一瞬。  
最初からcsvなどを読んで成形して・・・を行うコードを書かなくても良いので、一時的なセーブデータとしては重宝する！（日付のパースなどの処理をやり直さなくて良いので高速）

In [ ]:
kafun_renamed.to_pickle("kafun03.pkl")

# 現状のデータをplotしてみる

In [ ]:
kafun_renamed.京都府立医科大学.plot()

In [ ]:
kafun_renamed.plot(legend=False)

In [ ]:
kafun_renamed.tail()